# Intro & info

The mutational status of the IDH1 gene is a prognostic factor for gliomas, including glioblastomas (GBMs).

In this notebook, we will use radiomic features exctracted from MRI to classify glioblastomas as IDH1 wild-type (**IDH wt**) or mutant (**IDH mut**).


We will simulate a full pipeline, from image selection to model creation, using only free and publicly available resources.

We will use **HD-GLIO-AUTO** to obtain the segmentation of the lesions, **Pyradiomics** to extract the radiomics features, and **scikit-learn** to build our classifier.

While this notebook will not contain enough data to build a meaningful classifier, it should serve as a starting point to develop a fully functional one.

**Note**: for simplicity, we will refer to IDH1 as "IDH" when generating our dataframes.

#### Acknowledgments
This notebook was created by Dr. Gian Marco Conte from the **Mayo Clinic Radiology Informatics Lab** https://www.mayo.edu/research/labs/radiology-informatics/overview

with the help of Drs. Andrey Fedorov and Yashbir Singh.

# Before we start
This notebook is optimized to work on the **GCP platform**. (For the Colab version, see LINKTOCOLAB)

In [33]:
# Specify the project ID that points to your GCP project for billing purposes

#myProjectID="insert your project ID"
myProjectID = "idc-external-005"

# Import libraries

In [1]:
! pip install nibabel

     |████████████████████████████████| 3.3 MB 5.1 MB/s eta 0:00:01


In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)
%matplotlib inline

# Packages needed:

- dicomsort (https://dicomsort.com/documentation.html

- HD-GLIO-AUTO (https://github.com/NeuroAI-HD/HD-GLIO-AUTO)

- pydicom (https://pydicom.github.io/)

- pyradiomics (https://pyradiomics.readthedocs.io/en/latest/index.html)

### Install pydicom

In [8]:
!pip install pydicom

     |████████████████████████████████| 2.0 MB 5.2 MB/s eta 0:00:01


### Install dicomsort

In [9]:
#!git clone https://github.com/pieper/dicomsort.git
!python dicomsort/dicomsort.py --help


% dicomsort.py --help
dicomsort [options...] sourceDir targetDir/<patterns>

 where [options...] can be:
    [-z,--compressTargets] - create a .zip file in the target directory
    [-d,--deleteSource] - remove source files/directories after sorting
    [-f,--forceDelete] - remove source without confirmation
    [-k,--keepGoing] - report but ignore dupicate target files
    [-v,--verbose] - print diagnostics while processing
    [-s,--symlink] - create a symlink to dicom files in sourceDir instead of copying them
    [-t,--test] - run the built in self test (requires internet)
    [-u,--unsafe] - do not replace unsafe characters with '_' in the path
    [--help] - print this message

 where sourceDir is directory to be scanned or "" (null string) to read file list from stdin

 where targetDir/<patterns...> is a string defining the output file and directory
 names based on the dicom tags in the file.

If patterns are not specified, the following default is used:

  %PatientName-%Modalit

### Install HD-GLIO-AUTO

In [10]:
!docker pull jenspetersen/hd-glio-auto

Using default tag: latest
latest: Pulling from jenspetersen/hd-glio-auto

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: Pulling fs layer 
ac14c8e1: Pulling fs layer 
71a3f797: Pulling fs layer 
0ca295df: Pulling fs layer 
3b5df042: Pulling fs layer 
fa9bedc9: Pulling fs layer 
420927e2: Pulling fs layer 
0049843: Waiting fs layer 
6ab5e780: Pulling fs layer 
56b2329: Waiting fs layer 
bba74816: Pulling fs layer 
0c97a55: Waiting fs layer 
1c8b4073: Pulling fs layer 
9bc51944: Pulling fs layer 
c14c8e1: Waiting fs layer 
6f28461a: Pulling fs layer 
ad38789a: Pulling fs layer 
558b3aa0: Pulling fs layer 
Digest: sha256:401b2d8b0d4ab9ed1a2e7438790c81cb7de9e97f9548677cebc7cee0cc7f7b562KDownloading  52.04MB/250.1MBVerifying Checksum Downloading  631.1MB/1.913GBExtracting  1.157GB/1.913GBDownloading  985.2MB/5.543GBDownloading  641.6MB/2.827GBExtracting  4.221GB

### Install pyradiomics

In [9]:
!python -m pip install pyradiomics
!pyradiomics -h

usage: pyradiomics image|batch [mask] [Options]

optional arguments:
  -h, --help            show this help message and exit
  --label N, -l N       (DEPRECATED) Value of label in mask to use for
                        feature extraction.
  --version             Print version and exit

Input:
  Input files and arguments defining the extraction:
  - image and mask files (single mode) or CSV-file specifying them (batch mode)
  - Parameter file (.yml/.yaml or .json)
  - Overrides for customization type 3 ("settings")
  - Multi-threaded batch processing

  {Image,Batch}FILE     Image file (single mode) or CSV batch file (batch mode)
  MaskFILE              Mask file identifying the ROI in the Image. 
                        Only required when in single mode, ignored otherwise.
  --param FILE, -p FILE
                        Parameter file containing the settings to be used in extraction
  --setting "SETTING_NAME:VALUE", -s "SETTING_NAME:VALUE"
                        Additional parameters

# Query the TCGA-GBM cohort from the IDC platform using BigQuery
This cohort contains both pre- and post-surgery studies.
Since we are interested in the pre-surgery studies, the query will include only the earliest study for each subject.

We will save this information in a pandas dataframe called "tcga_gbm_mri" and refer to it as **imaging dataframe** in the comments.

In [11]:
%%bigquery tcga_gbm_mri


SELECT
    P.PatientID,
    P.StudyDate,
    P.StudyInstanceUID,
    P.gcs_url
FROM
    `canceridc-data.idc_views.dicom_all` AS P
WHERE
    P.collection_id = "tcga_gbm"
    AND
    P.Modality = "MR"
    AND
    P.StudyDate = 
    (
        SELECT
            MIN(C.StudyDate)
        FROM
            `canceridc-data.idc_views.dicom_all` AS C
        WHERE
            P.PatientID = C.PatientID
            AND
            C.collection_id = "tcga_gbm"
            AND
            C.Modality = "MR"
    )
ORDER BY
P.PatientID

Downloading: 100%|██████████| 249102/249102 [00:01<00:00, 197005.71rows/s]


#### Note: the gcs_url field contains the URL address for each DICOM file in this collection. We will use this information later in the notebook to download the scans.

In [12]:
# Let's print the dataframe
tcga_gbm_mri

,PatientID,StudyDate,StudyInstanceUID,gcs_url
0,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239/1.3.6.1.4.1.14519.5.2.1.1706.4001.220079143670418535732331991740/1.3.6.1.4.1.14519.5.2.1.1706.4001.155985359154963790784313803295.dcm#1592632444992521
1,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239/1.3.6.1.4.1.14519.5.2.1.1706.4001.220079143670418535732331991740/1.3.6.1.4.1.14519.5.2.1.1706.4001.292065088978667394712144965715.dcm#1592632445532611
2,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239/1.3.6.1.4.1.14519.5.2.1.1706.4001.251438991621054435135635316288/1.3.6.1.4.1.14519.5.2.1.1706.4001.182340981979083404495600128853.dcm#1592632446876833
3,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239/1.3.6.1.4.1.14519.5.2.1.1706.4001.999878355123004837026028734320/1.3.6.1.4.1.14519.5.2.1.1706.4001.172796066591829052357253758129.dcm#1592632452814857
4,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239/1.3.6.1.4.1.14519.5.2.1.1706.4001.240348717655601689187872263857/1.3.6.1.4.1.14519.5.2.1.1706.4001.236241315594671202210308608134.dcm#1592632446265443
...,...,...,...,...
249097,TCGA-76-6664,2002-01-10,1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790/1.3.6.1.4.1.14519.5.2.1.1188.4001.855165780560370170340861980177/1.3.6.1.4.1.14519.5.2.1.1188.4001.100343171081500912998461772831.dcm#1592632391555989
249098,TCGA-76-6664,2002-01-10,1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790/1.3.6.1.4.1.14519.5.2.1.1188.4001.182497160692414636856926373398/1.3.6.1.4.1.14519.5.2.1.1188.4001.180190222289499979980875796199.dcm#1592632376867576
249099,TCGA-76-6664,2002-01-10,1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790/1.3.6.1.4.1.14519.5.2.1.1188.4001.182497160692414636856926373398/1.3.6.1.4.1.14519.5.2.1.1188.4001.280823613153772212043085561017.dcm#1592632379595030
249100,TCGA-76-6664,2002-01-10,1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1188.4001.280508857811965887839758381790/1.3.6.1.4.1.14519.5.2.1.1188.4001.188264273641316994799482740951/1.3.6.1.4.1.14519.5.2.1.1188.4001.290093504099011504768489851011.dcm#1592632380991315


#### Now let's check how many patients and how many exams we have

In [13]:
num_subjects = len(tcga_gbm_mri["PatientID"].unique())
num_studies = len(tcga_gbm_mri["StudyInstanceUID"].unique())
print(f"Total number of subjects in old approach: {num_subjects}")
print(f"Total number of studies in old approach: {num_studies}")

Total number of subjects in old approach: 259
Total number of studies in old approach: 284


Some subjects have multiple studies performed the same day, so we have more studies than subjects.

There is no programmatic way of knowing which study to include, so the only option will be to manually review these cases and select the ones to include (we will do this later in the notebook).

# Match imaging and genomic information.

We obtained the IDH1 mutational status information from the Genomic Data Commons portal (GDC) https://portal.gdc.cancer.gov/).

You can find the spreadsheet with the genomic information "IDH_mutant_TCGA-GBM.csv" here LINK. The subjects included in this spreadsheet are the ones in which a mutation in the IDH1 gene was detected; hence, they can be included in the "IDH mutant" class.

To date, manually uploading the spreadsheet is the only way to match genomic and imaging data.

For more info see https://datascience.cancer.gov/news-events/blog/cancer-data-aggregator-engine-could-drive-data-aggregation-whole-new-way

In [15]:
# Load the file with the genomic information
idh_df = pd.read_csv('IDH_mutant_TCGA-GBM.csv')
idh_df

,Case ID,Project,Gender,Age at diagnosis,Days to death,Vital Status,Primary Diagnosis
0,TCGA-12-0818,TCGA-GBM,Female,45 years 230 days,7 years 235 days,Dead,Glioblastoma
1,TCGA-12-1088,TCGA-GBM,Female,53 years 309 days,10 years 229 days,Dead,Glioblastoma
2,TCGA-16-1460,TCGA-GBM,Female,36 years 59 days,--,Alive,Glioblastoma
3,TCGA-26-1442,TCGA-GBM,Male,43 years 245 days,--,Alive,Glioblastoma
4,TCGA-06-5417,TCGA-GBM,Female,45 years 136 days,--,Alive,Glioblastoma
5,TCGA-14-1456,TCGA-GBM,Male,23 years 310 days,--,Alive,Glioblastoma
6,TCGA-06-6701,TCGA-GBM,Male,60 years 136 days,--,Alive,Glioblastoma
7,TCGA-27-2521,TCGA-GBM,Male,34 years 267 days,1 year 145 days,Dead,Glioblastoma
8,TCGA-06-1805,TCGA-GBM,Female,28 years 315 days,--,Alive,Glioblastoma
9,TCGA-06-2570,TCGA-GBM,Female,21 years 266 days,--,Alive,Glioblastoma


The "idh_df" dataframe contains the PatientIDs ('Case ID') of the IDH1 mutant subjects. We will refer to this dataframe as **genomic dataframe**.

Since MRI data is not available for all the subjects of the GDC TCGA-GBM cohort, we will first compare the imaging and genomic dataframes, and then add the IDH1 mutational status when available ("IDH Status").

In [16]:
# First let's make a copy of the original dataframe
tcga_gbm_mri_idh = tcga_gbm_mri.copy()

#### Let's check how many subjects of the genomic dataframe are missing from the imaging dataframe.

In [17]:
# Finds the differences between the IDs columns of the two dataframes
missing_subjects = set(idh_df['Case ID']).difference(set(tcga_gbm_mri_idh['PatientID']))

print(f"A total of {len(missing_subjects)} subjects are missing from the imaging dataframe.")
print("The missing subjects are: "+ str(missing_subjects))

A total of 13 subjects are missing from the imaging dataframe.
The missing subjects are: {'TCGA-26-1442', 'TCGA-16-0850', 'TCGA-12-1088', 'TCGA-16-0849', 'TCGA-12-0827', 'TCGA-16-1460', 'TCGA-32-4208', 'TCGA-12-0818', 'TCGA-02-2483', 'TCGA-06-1805', 'TCGA-19-A6J5', 'TCGA-06-A7TL', 'TCGA-14-4157'}


#### Now let's add the IDH status information for the subjects that are present in both dataframes.


In [18]:
# Add '1' to all images corresponding to an IDH mutant subject and '0' to IDH wt
tcga_gbm_mri_idh['IDH Status'] = tcga_gbm_mri_idh['PatientID'].isin(idh_df['Case ID']).astype(int)

In [19]:
# List all the images for the IDH mutant subjects
tcga_gbm_mri_idh.loc[tcga_gbm_mri_idh['IDH Status'] == 1]

,PatientID,StudyDate,StudyInstanceUID,gcs_url,IDH Status
38896,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.244190093169259767085353590019.dcm#1592634419703252,1
38897,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.258624254650871593065381246886/1.3.6.1.4.1.14519.5.2.1.4591.4001.178150653834502649828798714547.dcm#1592636558729327,1
38898,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.483064039435491061849534492565.dcm#1592634421586492,1
38899,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.575709654718829638269159655776/1.3.6.1.4.1.14519.5.2.1.4591.4001.298258654588370812068474314303.dcm#1592636564834517,1
38900,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.264367681056898400247780398742.dcm#1592634420068875,1
...,...,...,...,...,...
232097,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.401946651009042036823821396148.dcm#1592633475299120,1
232098,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.739107246764853439484568258615.dcm#1592633475396448,1
232099,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.273440342697389360017292839411.dcm#1592633474607545,1
232100,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.101799618749764475591299151247.dcm#1592633473557341,1


In [20]:
# Let's visualize only the PatientID and the StudyDate of the IDH mutant class
tcga_gbm_mri_idh.loc[tcga_gbm_mri_idh['IDH Status'] == 1].drop(['gcs_url','StudyInstanceUID'],axis=1).drop_duplicates(['PatientID','StudyDate'])

,PatientID,StudyDate,IDH Status
38896,TCGA-06-0128,1999-02-18,1
40673,TCGA-06-0129,2000-03-14,1
95028,TCGA-06-0221,1998-06-30,1
134124,TCGA-06-2570,2007-07-26,1
146347,TCGA-06-5417,2008-09-03,1
149936,TCGA-06-6389,2009-04-04,1
153634,TCGA-06-6701,2009-06-06,1
189944,TCGA-14-0871,1991-07-05,1
191987,TCGA-14-1456,1999-04-22,1
192291,TCGA-14-1458,2000-06-25,1


# Know your data

Datasets are rarely ready to use to train your models; it is fundamental to perform quality control of the data you are going to use, or you will end up with a useless model.

For example, the cohort we are using contains more than one study per subject, and within each study, there could be repeated scans (usually because of motion artefacts; for example see TCGA-08-0522). Moreover, some of the cases do not have a pre-surgery scan, so they will need to be excluded from the analysis.

It is important to be aware of these details if you are planning to randomly select a subset of patients from a given cohort, as the following example will show.

# Select random cases

We will now select 5 subjects for each class:

- IDH Status = 0 (IDH wild type; **IDH wt**)
- IDH Status = 1 (IDH mutant; **IDH mut**)

We will select only 5 subjects per class to reduce downloading and computing time; in reality, you will need much more data to train a meaningful classifier.

To include more patients, change the value of the "N" variable (remember: in this cohort, we have a total of 15 patients with the IDH mutation).

In [21]:
N = 5
random_selection = tcga_gbm_mri_idh[['PatientID','IDH Status']].drop_duplicates(['PatientID']).groupby('IDH Status').apply(lambda x:x.sample(N, random_state=33)).reset_index(drop=True)
random_selection

,PatientID,IDH Status
0,TCGA-19-5956,0
1,TCGA-06-0213,0
2,TCGA-08-0350,0
3,TCGA-08-0522,0
4,TCGA-08-0358,0
5,TCGA-06-0128,1
6,TCGA-19-2629,1
7,TCGA-06-0129,1
8,TCGA-14-1821,1
9,TCGA-27-2521,1


In [22]:
m = tcga_gbm_mri_idh.PatientID.isin(random_selection.PatientID) #use the random selection as a "filter" on the main dataframe

final_df = tcga_gbm_mri_idh[m]
final_df 

,PatientID,StudyDate,StudyInstanceUID,gcs_url,IDH Status
38896,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.244190093169259767085353590019.dcm#1592634419703252,1
38897,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.258624254650871593065381246886/1.3.6.1.4.1.14519.5.2.1.4591.4001.178150653834502649828798714547.dcm#1592636558729327,1
38898,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.483064039435491061849534492565.dcm#1592634421586492,1
38899,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.575709654718829638269159655776/1.3.6.1.4.1.14519.5.2.1.4591.4001.298258654588370812068474314303.dcm#1592636564834517,1
38900,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.264367681056898400247780398742.dcm#1592634420068875,1
...,...,...,...,...,...
232097,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.401946651009042036823821396148.dcm#1592633475299120,1
232098,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.739107246764853439484568258615.dcm#1592633475396448,1
232099,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.273440342697389360017292839411.dcm#1592633474607545,1
232100,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458/1.3.6.1.4.1.14519.5.2.1.3775.4001.123762435054161578259889405532/1.3.6.1.4.1.14519.5.2.1.3775.4001.101799618749764475591299151247.dcm#1592633473557341,1


In [23]:
# Get the PatientID of the subjects included
final_df['PatientID'].unique()

array(['TCGA-06-0128', 'TCGA-06-0129', 'TCGA-06-0213', 'TCGA-08-0350',
       'TCGA-08-0358', 'TCGA-08-0522', 'TCGA-14-1821', 'TCGA-19-2629',
       'TCGA-19-5956', 'TCGA-27-2521'], dtype=object)

# Verify the completeness of the studies

In this notebook, we will extract radiomics features from T2- and post-contrast T1-weighted images.

To obtain the segmentations of the lesions, we will use HD-GLIO-AUTO, which needs four  axial sequences as inputs:
- pre-contrast T1
- post-contrast T1
- T2
- FLAIR

Although we could write a script that programmatically selects the series that we need using DICOM tags, in reality, the heterogeneity of the naming of the sequences makes it hardly feasible. Therefore we suggest to visually inspect your data both for quality control and series selection.

#### To inspect the selected subjects, you can either use the IDC portal (https://portal.imaging.datacommons.cancer.gov/) or retrieve the StudyInstanceUID information from the dataframe we generated and run the following cells:

In [24]:
def get_idc_viewer_url(StudyInstanceUID):
    return "https://viewer.imaging.datacommons.cancer.gov/viewer/"+StudyInstanceUID

In [25]:
# Follow the link after printing. Change the StudyInstanceUID as needed.
print(get_idc_viewer_url("1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909"))

https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909


### We found the following issues for the randomly selected subjects:

For the **IDH wt** subgroup:
- TCGA-19-5956: does not have all the necessary sequences
- TCGA-08-0350: does not have all the necessary sequences
- TCGA-08-0522: does not have all the necessary sequences
- TCGA-08-0358: the necessary sequences needs to be retrieved from two different studies performed on the same day

For the **IDH mut** subgroup:
- TCGA-19-2629: does not have all the necessary sequences
- TCGA-14-1821: only post-surgery scans
- TCGA-27-2521: does not have all the necessary sequences

### As you can see, different issues can be encountered during quality control.

Subjects that do not have all the MRI sequences necessary for the radiomics analysis will need to be excluded.

For the subjects in which the MRI sequences needed to run the segmentation model HD-GLIO-AUTO are missing, we have two options:

- Obtain the segmentations manually or using other models
- Find a way to substitute the missing scans

Regarding the second option, we could use a generative adversarial network to generate the missing MRI sequences; you can find a paper authored by our group describing this approach here: https://pubs.rsna.org/doi/full/10.1148/radiol.2021203786

# Select complete MRI studies

#### After reviewing the cohort, we were able to identify subjects that fullfills the inclusion criteria.

#### You can either use the same subjects we identified, or explore the cohort and find others.

In [24]:
# Lists of subjects that fulfils the inclusion criteria
IDH_wt = ["TCGA-06-0213", "TCGA-08-0358", "TCGA-02-0033", "TCGA-02-0034", "TCGA-02-0037"]
IDH_mut = ["TCGA-06-0128", "TCGA-06-0129", "TCGA-06-2570", "TCGA-06-5417", "TCGA-06-6389"]
final_list = IDH_mut + IDH_wt

In [38]:
# Lists of subjects that fulfils the inclusion criteria
IDH_wt = ["TCGA-02-0033"]
IDH_mut = ["TCGA-06-0128"]
final_list = IDH_mut + IDH_wt

#### Let's update our dataframe with the selected patients

In [40]:
m = tcga_gbm_mri_idh.PatientID.isin(final_list)

final_df = tcga_gbm_mri_idh[m]
final_df

,PatientID,StudyDate,StudyInstanceUID,gcs_url,IDH Status
2813,TCGA-02-0033,1997-05-26,1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.250526316070055292979086457976/1.3.6.1.4.1.14519.5.2.1.1706.4001.323119290918952874626865868030.dcm#1592632595605510,0
2814,TCGA-02-0033,1997-05-26,1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.105844169858907809389536619529/1.3.6.1.4.1.14519.5.2.1.1706.4001.326535645810693359307312583401.dcm#1592632594735390,0
2815,TCGA-02-0033,1997-05-26,1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.874349474095633498635576824961/1.3.6.1.4.1.14519.5.2.1.1706.4001.214162620943130559794867273793.dcm#1592632597684401,0
2816,TCGA-02-0033,1997-05-26,1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.250526316070055292979086457976/1.3.6.1.4.1.14519.5.2.1.1706.4001.272158003279744043259572278361.dcm#1592632595507623,0
2817,TCGA-02-0033,1997-05-26,1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.105844169858907809389536619529/1.3.6.1.4.1.14519.5.2.1.1706.4001.189549074915981088674484700653.dcm#1592632594637084,0
...,...,...,...,...,...
40668,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.258624254650871593065381246886/1.3.6.1.4.1.14519.5.2.1.4591.4001.145326100447495733962560765218.dcm#1592636558528394,1
40669,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.270271453584776537035782480315/1.3.6.1.4.1.14519.5.2.1.4591.4001.247049290955896150038266040236.dcm#1592636559825286,1
40670,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.258624254650871593065381246886/1.3.6.1.4.1.14519.5.2.1.4591.4001.146343257946831277020701913154.dcm#1592636558559038,1
40671,TCGA-06-0128,1999-02-18,1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.575709654718829638269159655776/1.3.6.1.4.1.14519.5.2.1.4591.4001.438274313075488492049664896210.dcm#1592636565813031,1


In [41]:
final_df['PatientID'].unique()

array(['TCGA-02-0033', 'TCGA-06-0128'], dtype=object)

# Download and sort imaging data.

#### We will generate two different URL lists based on the IDH status.

For details on downloading data to your notebook, see https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/Cohort_download.ipynb 

In [42]:
# List of IDH wild type cases
IDH_wt_url = final_df.loc[final_df['IDH Status'] == 0]
IDH_wt_url = IDH_wt_url.join(IDH_wt_url["gcs_url"].str.split('#', 1, expand=True).rename(columns={0:'gcs_url_no_revision', 1:'gcs_revision'}))
IDH_wt_url["gcs_url_no_revision"].to_csv("gcs_paths_idh_wt.txt", header=False, index=False)

# List of IDH mutant cases
IDH_mut_url = final_df.loc[final_df['IDH Status'] == 1]
IDH_mut_url = IDH_mut_url.join(IDH_mut_url["gcs_url"].str.split('#', 1, expand=True).rename(columns={0:'gcs_url_no_revision', 1:'gcs_revision'}))
IDH_mut_url["gcs_url_no_revision"].to_csv("gcs_paths_idh_mut.txt", header=False, index=False)

In [43]:
!head gcs_paths_idh_mut.txt

gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.244190093169259767085353590019.dcm
gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.258624254650871593065381246886/1.3.6.1.4.1.14519.5.2.1.4591.4001.178150653834502649828798714547.dcm
gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.197146947506099366832803736438/1.3.6.1.4.1.14519.5.2.1.4591.4001.393766610979582366190694521912/1.3.6.1.4.1.14519.5.2.1.4591.4001.483064039435491061849534492565.dcm
gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.796093665645495657978672735909/1.3.6.1.4.1.14519.5.2.1.4591.4001.575709654718829638269159655776/1.3.6.1.4.1.14519.5.2.1.4591.4001.298258654588370812068474314303.dcm
gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.1971469475060993668328037

### Download the DICOM files to our virtual machine

In [44]:
!mkdir downloaded_cohort
!cat gcs_paths_idh_wt.txt | gsutil -u $myProjectID -m cp -Ir ./downloaded_cohort
!cat gcs_paths_idh_mut.txt | gsutil -u $myProjectID -m cp -Ir ./downloaded_cohort

Copying gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.250526316070055292979086457976/1.3.6.1.4.1.14519.5.2.1.1706.4001.323119290918952874626865868030.dcm...
Copying gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.105844169858907809389536619529/1.3.6.1.4.1.14519.5.2.1.1706.4001.326535645810693359307312583401.dcm...
Copying gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.874349474095633498635576824961/1.3.6.1.4.1.14519.5.2.1.1706.4001.214162620943130559794867273793.dcm...
Copying gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/1.3.6.1.4.1.14519.5.2.1.1706.4001.250526316070055292979086457976/1.3.6.1.4.1.14519.5.2.1.1706.4001.272158003279744043259572278361.dcm...
Copying gs://idc-tcia-tcga-gbm/dicom/1.3.6.1

### Sort the downloaded DICOM files using dicomosort

The DICOM images will be copied and sorted in the folder "cohort_sorted"

In [45]:
!python dicomsort/dicomsort.py -u downloaded_cohort cohort_sorted/%PatientID/%StudyDate/%StudyInstanceUID/%SeriesDescription/%SeriesInstanceUID/%SOPInstanceUID.dcm

100%|██████████████████████████████████████| 2210/2210 [00:03<00:00, 685.96it/s]
Files sorted


# Obtain the segmentations using HD-GLIO-AUTO

HD-GLIO-AUTO is the result of a joint project between the Department of Neuroradiology at the Heidelberg University Hospital, Germany and the Division of Medical Image Computing at the German Cancer Research Center (DKFZ) Heidelberg, Germany.

For more information see https://github.com/NeuroAI-HD/HD-GLIO-AUTO

As explained earlier, it requires four MRI sequence to obtain the segmentation of the lesions.

HD-GLIO-AUTO can work directly with DICOM files, but requires each folder to follow a precise organization and naming scheme (see gitHub page for more details).

Given the heterogeneity of the studies, we will opt for a manual cleaning and organization of the folders, which consist in deleting unnecessary studies and rename the folders using the following naming scheme:

- T1: for folders containing pre-contrast T1 images
- CT1: for folders containing post-contrast T1 images
- T2: for folders containing T2 images
- FLAIR: for folders containing FLAIR images

#### Notes from data cleaning:

- TCGA-06-0128: this subject has two identical studies performed the same day, with different StudyInstanceUID but same acquisition time; we can safely get rid of one of the studies.
- TCGA-06-0128: this subject has two post-contrast T1 series, one of which contains only 4 slices; since it is unclear why this serie was acquired, or what it represents, we can delete it for the purpose of this notebook.
- TCGA-06-0128: the T2 sequence is a dual echo FSE (http://mriquestions.com/dual-echo-fse.html); will it work with HD-glio?
- TCGA-06-0129: (one of) the post-contrast T1 serie has no series description tag

TO DO: provide cleaned examples to avoid the manual cleaning process to the users

# DEV

Temporary (ugly) workaround the cleaning process

In [46]:
os.getcwd()

'/home/jupyter'

In [47]:
os.chdir('./cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744')

In [48]:
# temporary workaround the data cleaning process
#os.listdir('./cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744')
os.listdir()

['SAG T1 POST',
 'COR T1 POST',
 'AX FLAIR',
 '3D WAND T1 WEIGHTED',
 'Diffusion Weighted',
 'AX T2 FSE',
 'LOC',
 'AX T1',
 '2D WAND T2 WEIGHTED',
 'AX T1 POST']

In [49]:
os.rename('AX T1','T1')
os.rename('AX T1 POST','CT1')
os.rename('AX T2 FSE','T2')
os.rename('AX FLAIR','FLAIR')

In [50]:
shutil.rmtree('SAG T1 POST')
shutil.rmtree('2D WAND T2 WEIGHTED')
shutil.rmtree('LOC')
shutil.rmtree('Diffusion Weighted')
shutil.rmtree('COR T1 POST')
shutil.rmtree('3D WAND T1 WEIGHTED')

In [51]:
os.mkdir('segmentation_output')

In [52]:
os.listdir()

['segmentation_output', 'T2', 'T1', 'FLAIR', 'CT1']

In [53]:
os.getcwd()

'/home/jupyter/cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744'

In [54]:
# get back to the main directori
os.chdir('/home/jupyter')

In [55]:
os.getcwd()

'/home/jupyter'

### Now let's run a test using HD-GLIO-AUTO

#### Here is the general command to run HD-GLIO-AUTO
docker run --mount type=bind,source=YOUR_INPUT_FOLDER,target=/input --mount type=bind,source=YOUR_OUTPUT_FOLDER,target=/output jenspetersen/hd-glio-auto

We will replace "docker run" with "nvidia-docker run"; for more details see https://github.com/NeuroAI-HD/HD-GLIO-AUTO

In [56]:
!nvidia-docker run --mount type=bind,source="/home/jupyter/cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744",target=/input --mount type=bind,source="/home/jupyter/cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/segmentation_output",target=/output jenspetersen/hd-glio-auto

09:30:23 PM: Reading series 1.3.6.1.4.1.14519.5.2.1.1706.4001.103174687731052142735983046836
09:30:23 PM: Reading series 1.3.6.1.4.1.14519.5.2.1.1706.4001.287753826073472752590065451465
09:30:23 PM: Reading series 1.3.6.1.4.1.14519.5.2.1.1706.4001.261091685370401963952512175444
09:30:23 PM: Reading series 1.3.6.1.4.1.14519.5.2.1.1706.4001.116289242878713280880004720679
THCudaCheck FAIL file=/pytorch/aten/src/THC/THCGeneral.cpp line=47 error=999 : unknown error
Traceback (most recent call last):
  File "/usr/local/bin/hd-bet", line 7, in <module>
    exec(compile(f.read(), __file__, 'exec'))
  File "/HD-BET/HD_BET/hd-bet", line 119, in <module>
    run_hd_bet(input_files, output_files, mode, config_file, device, pp, tta, save_mask, overwrite_existing)
  File "/HD-BET/HD_BET/run.py", line 63, in run_hd_bet
    net.cuda(device)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 458, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "/usr/local/l

#### Check the segmentation (TODO: switch to itkwidgets viewer)

In [ ]:
T2_mri = nib.load('/home/jupyter/cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/segmentation_output/T2_r2s_bet_reg.nii.gz').get_fdata()
T1Gd_mri = nib.load('/home/jupyter/cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/segmentation_output/CT1_r2s_bet_reg.nii.gz').get_fdata()
segmentation_mask = nib.load('/home/jupyter/cohort_sorted/TCGA-02-0033/19970526/1.3.6.1.4.1.14519.5.2.1.1706.4001.211135800355624542661804589744/segmentation_output/segmentation.nii.gz').get_fdata()

In [ ]:
# Make zero values in the mask as nan for best visualization
segmentation_mask[segmentation_mask == 0.0] = np.nan

# rotate the image
T2_mri_rot = np.rot90(T2_mri, 1)
T1Gd_mri_rot = np.rot90(T1Gd_mri, 1)
segmentation_mask_rot = np.rot90(segmentation_mask, 1)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(T2_mri_rot[:,:,15],cmap="gray")
plt.imshow(segmentation_mask_rot[:,:,15],cmap='bwr',alpha=0.2)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(T1Gd_mri_rot[:,:,15],cmap="gray")
plt.imshow(segmentation_mask_rot[:,:,15],cmap='bwr',alpha=0.1)